<a href="https://colab.research.google.com/github/tamara-kostova/IIS/blob/master/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ndlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.2/110.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.1 MB/s eta 0:00:00


# **Exercise 1**

Network Definition

In [20]:
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.opinions as op

N = 1000
p = 0.1
g = nx.erdos_renyi_graph(N, p)

Model definition

In [22]:
models = {
    'SI': ep.SIModel(g),
    'SIS': ep.SISModel(g),
    'SEIS': ep.SEISModel(g),
    'SEIR': ep.SEIRModel(g)
}
for model_name, model_instance in models.items():
    print(f"Model: {model_name}")
    print(f"Parameters: {model_instance.parameters}")
    print(f"Available Statuses: {model_instance.available_statuses}")
    print("\n")

Model: SI
Parameters: {'model': {'beta': {'descr': 'Infection rate', 'range': '[0,1]', 'optional': False}, 'tp_rate': {'descr': 'Whether if the infection rate depends on the number of infected neighbors', 'range': [0, 1], 'optional': True, 'default': 1}}, 'nodes': {}, 'edges': {}}
Available Statuses: {'Susceptible': 0, 'Infected': 1}


Model: SIS
Parameters: {'model': {'beta': {'descr': 'Infection rate', 'range': [0, 1], 'optional': False}, 'lambda': {'descr': 'Recovery rate', 'range': [0, 1], 'optional': False}, 'tp_rate': {'descr': 'Whether if the infection rate depends on the number of infected neighbors', 'range': [0, 1], 'optional': True, 'default': 1}}, 'nodes': {}, 'edges': {}}
Available Statuses: {'Susceptible': 0, 'Infected': 1}


Model: SEIS
Parameters: {'model': {'alpha': {'descr': 'Incubation period', 'range': [0, 1], 'optional': False}, 'beta': {'descr': 'Infection rate', 'range': [0, 1], 'optional': False}, 'lambda': {'descr': 'Recovery rate', 'range': [0, 1], 'optional':

Set parameters

In [6]:
parameters = {
    'SI': {'beta': 0.001, 'fraction_infected': 0.05},
    'SIS': {'beta': 0.001, 'lambda': 0.01, 'fraction_infected': 0.05},
    'SEIS': {'beta': 0.001, 'lambda': 0.01, 'alpha': 0.005, 'fraction_infected': 0.05},
    'SEIR': {'beta': 0.001, 'gamma': 0.01, 'alpha': 0.005, 'fraction_infected': 0.05}
}

In [8]:
pip install pyqt5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.1/338.1 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 14.7 MB/s eta 0:00:00


Simulations and visualizations

In [23]:
import ndlib.models.ModelConfig as mc
import matplotlib
# matplotlib.use('Agg')
import os
import logging
import matplotlib.pyplot as plt
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

if os.environ.get('DISPLAY', '') == '':
    logging.warning('No display found. Using non-interactive Agg backend.')
    matplotlib.use('Agg')
    for model_name, model in models.items():
        print(model)
        config = mc.Configuration()
        params = parameters[model_name]
        for param, value in params.items():
            config.add_model_parameter(param, value)
        print(config)
        model.set_initial_status(config)
        iterations = model.iteration_bunch(200)
        trends = model.build_trends(iterations)

        print(f"{model_name} simulation done.")

        print("Iterations:", iterations)
        print("Trends data:", trends)

        viz = DiffusionTrend(model, trends)
        viz.plot()
        plt.title(f"Diffusion Trend for {model_name} Model")
        plt.show()
        plt.close()


SI simulation done.
Iterations: [{'iteration': 0, 'status': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 0, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 1, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 1

# **Exercise 2**

Network definition

In [29]:
N = 1000
p = 0.1
g = nx.erdos_renyi_graph(N, p)

Model definition

In [30]:
models = {
    'Voter': op.VoterModel(g),
    'QVoter': op.QVoterModel(g),
    'MajorityRuleModel': op.MajorityRuleModel(g),
    'Sznajd': op.SznajdModel(g)
}
for model_name, model_instance in models.items():
    print(f"Model: {model_name}")
    print(f"Parameters: {model_instance.parameters}")
    print(f"Available Statuses: {model_instance.available_statuses}")
    print("\n")

Model: Voter
Parameters: {'model': {}, 'nodes': {}, 'edges': {}}
Available Statuses: {'Susceptible': 0, 'Infected': 1}


Model: QVoter
Parameters: {'model': {'q': {'descr': 'Number of neighbours that affect the opinion of an agent', 'range': [0, 1000], 'optional': False}}, 'nodes': {}, 'edges': {}}
Available Statuses: {'Susceptible': 0, 'Infected': 1}


Model: MajorityRuleModel
Parameters: {'model': {'q': {'descr': 'Number of randomly chosen voters', 'range': [0, 1000], 'optional': False}}, 'nodes': {}, 'edges': {}}
Available Statuses: {'Susceptible': 0, 'Infected': 1}


Model: Sznajd
Parameters: {'model': {}, 'nodes': {}, 'edges': {}}
Available Statuses: {'Susceptible': 0, 'Infected': 1}




Simulation and visualization

In [32]:
for model_name, model in models.items():
    config = mc.Configuration()

    config.add_model_parameter('q', 4)

    model.set_initial_status(config)

    iterations = model.iteration_bunch(200)
    trends = model.build_trends(iterations)

    print(f"{model_name} simulation done.")
    print("Iterations:", iterations)

    viz = DiffusionTrend(model, trends)
    viz.plot()
    plt.title(f"Diffusion Trend for {model_name} Model")
    plt.show()
    plt.close()

Voter simulation done.
Iterations: [{'iteration': 200, 'status': {402: 0}, 'node_count': {0: 901, 1: 99}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 201, 'status': {252: 1}, 'node_count': {0: 900, 1: 100}, 'status_delta': {0: -1, 1: 1}}, {'iteration': 202, 'status': {669: 0}, 'node_count': {0: 901, 1: 99}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 203, 'status': {413: 0}, 'node_count': {0: 901, 1: 99}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 204, 'status': {855: 0}, 'node_count': {0: 902, 1: 98}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 205, 'status': {616: 0}, 'node_count': {0: 902, 1: 98}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 206, 'status': {886: 0}, 'node_count': {0: 902, 1: 98}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 207, 'status': {178: 0}, 'node_count': {0: 903, 1: 97}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 208, 'status': {922: 0}, 'node_count': {0: 903, 1: 97}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 209, 'status': {137: 0}, 'node_

# **Exercise 3**

Graph configuration

In [34]:
g = nx.read_edgelist("drive/MyDrive/facebook_combined.txt", create_using=nx.Graph(), nodetype=int)

In [35]:
g

Models

In [36]:
models = {
    'Voter': op.VoterModel(g),
    'QVoter': op.QVoterModel(g),
    'MajorityRule': op.MajorityRuleModel(g),
    'Sznajd': op.SznajdModel(g)
}

Set parameters

In [43]:
parameters = {
    'Voter': {'percentage_infected': 0.1},
    'QVoter': {'percentage_infected': 0.1, 'q': 3},
    'MajorityRule': {'percentage_infected': 0.1, 'q': 3},
    'Sznajd': {'percentage_infected': 0.1}
}

Simulation

In [44]:
for model_name, model_instance in models.items():
    print(f"Model: {model_name}")
    config = mc.Configuration()
    for param, value in parameters[model_name].items():
        config.add_model_parameter(param, value)
    model_instance.set_initial_status(config)
    iterations = model_instance.iteration_bunch(200)
    trends = model_instance.build_trends(iterations)
    print(f"Iterations: {iterations}")
    print(f"Trends: {trends}")

Model: Voter
Iterations: [{'iteration': 400, 'status': {1951: 0}, 'node_count': {0: 2833, 1: 1206}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 401, 'status': {1756: 0}, 'node_count': {0: 2833, 1: 1206}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 402, 'status': {2546: 1}, 'node_count': {0: 2833, 1: 1206}, 'status_delta': {0: -1, 1: 1}}, {'iteration': 403, 'status': {1339: 0}, 'node_count': {0: 2833, 1: 1206}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 404, 'status': {1770: 0}, 'node_count': {0: 2834, 1: 1205}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 405, 'status': {3943: 1}, 'node_count': {0: 2833, 1: 1206}, 'status_delta': {0: -1, 1: 1}}, {'iteration': 406, 'status': {852: 1}, 'node_count': {0: 2833, 1: 1206}, 'status_delta': {0: -1, 1: 1}}, {'iteration': 407, 'status': {2743: 1}, 'node_count': {0: 2833, 1: 1206}, 'status_delta': {0: -1, 1: 1}}, {'iteration': 408, 'status': {751: 0}, 'node_count': {0: 2834, 1: 1205}, 'status_delta': {0: 1, 1: -1}}, {'iteration': 409, 'st

Visualization

In [45]:
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
for model_name, model_instance in models.items():
    viz = DiffusionTrend(model_instance, trends)
    viz.plot()
    plt.title(f"Diffusion Trend for {model_name} Model")
    plt.show()
    plt.close()